In [24]:
import os
import glob
import pandas as pd
import numpy as np
from collections import defaultdict
import copy
import subprocess
import cv2
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from tqdm import tqdm
import sys

# Set the working directory to the notebook location
notebook_dir = os.getcwd()  # Get current working directory of the notebook

# Ensure custom module paths are included
tracking_diagnostics_path = os.path.abspath(os.path.join(notebook_dir, 'tracking-diagnostics'))

sys.path.append(tracking_diagnostics_path)

print("Tracking diagnostics path:", tracking_diagnostics_path)

from diagnostics.video import get_frames_from_idxs
from eks.utils import convert_lp_dlc, format_data
from pseudo_labeler.utils import format_data_walk
from pseudo_labeler.evaluation import compute_likelihoods_and_variance, plot_heatmaps

Tracking diagnostics path: /teamspace/studios/this_studio/tracking-diagnostics


In [25]:
# ADJUST THESE

dir = '/teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000'
videos_dir = '/teamspace/studios/this_studio/data/mirror-mouse'  # video to be overlayed on
n_frames = 10
top_error_frame_count = 3
excluded_keypoints = ['obs_top', 'obsHigh_bot', 'obsLow_bot']

# Define paths relative to the base directory
networks_dir = os.path.join(dir, 'networks')
eks_dir = os.path.join(dir, 'post-processors', 'eks_rng=0-3')  # specify this directory
aeks_dir = os.path.join(dir, 'results_amortized_eks')
aeks_eks_dir = os.path.join(aeks_dir, 'eks_rng=0-3')  # New directory
output_dir = os.path.join(dir, 'eks_aeks_pixel_diff')
relative_pixel_error_dir = os.path.join(output_dir, 'relative_pixel_error')
specifier = os.path.basename(dir)

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

# Print paths to verify
print(f"Base Directory: {dir}")
print(f"Videos Directory: {videos_dir}")
print(f"Number of Frames: {n_frames}")
print(f"Excluded Keypoints: {excluded_keypoints}")
print(f"Networks Directory: {networks_dir}")
print(f"EKS Directory: {eks_dir}")
print(f"AEKS Directory: {aeks_dir}")
print(f"AEKS EKS Directory: {aeks_eks_dir}")
print(f"Output Directory: {output_dir}")
print(f"Relative Pixel Error Directory: {relative_pixel_error_dir}")
print(f"Specifier: {specifier}")

Base Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000
Videos Directory: /teamspace/studios/this_studio/data/mirror-mouse
Number of Frames: 10
Excluded Keypoints: ['obs_top', 'obsHigh_bot', 'obsLow_bot']
Networks Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/networks
EKS Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/post-processors/eks_rng=0-3
AEKS Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/results_amortized_eks
AEKS EKS Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/results_amortized_eks/eks_rng=0-3
Output Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/eks_aeks_pixel_diff
Relative Pixel Error Directory: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/eks_aeks_pixel_diff/relative_pixel_error
Specifier: hand=100_pseudo=1000


In [26]:
# Helper Functions
def get_subdirectories(directory):
    return [os.path.join(directory, sub_dir) for sub_dir in os.listdir(directory) if os.path.isdir(os.path.join(directory, sub_dir))]

def get_csv_files(directory):
    return glob.glob(os.path.join(directory, '**/*.csv'), recursive=True)

def group_csv_files(eks_files, aeks_files):
    file_groups = defaultdict(list)
    
    for file in eks_files:
        file_name = os.path.basename(file)
        file_groups[file_name].append(file)
        
    for file in aeks_files:
        file_name = os.path.basename(file)
        file_groups[file_name].append(file)
        
    return file_groups

def calculate_pixel_error(file1, file2, top_frame_count=20):
    df1 = pd.read_csv(file1, header=[0, 1, 2])
    df2 = pd.read_csv(file2, header=[0, 1, 2])
    
    scorer1 = df1.columns.levels[0][0]  # Automatically detect the first-level column header for df1
    scorer2 = df2.columns.levels[0][0]  # Automatically detect the first-level column header for df2
    
    keypoints = df1.columns.get_level_values(1).unique()
    errors = defaultdict(list)
    top_error_frames = []
    
    for keypoint in keypoints:
        if keypoint == 'bodyparts':
            continue
        
        x1 = df1[scorer1, keypoint, 'x']
        y1 = df1[scorer1, keypoint, 'y']
        x2 = df2[scorer2, keypoint, 'x']
        y2 = df2[scorer2, keypoint, 'y']
        
        distance = np.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)
        errors[keypoint] = distance
        
        top_indices = distance.nlargest(top_frame_count).index
        top_error_frames.extend([(keypoint, idx, distance[idx]) for idx in top_indices])
    
    error_df = pd.DataFrame(errors)
    top_error_df = pd.DataFrame(top_error_frames, columns=['keypoint', 'frame', 'error'])
    return error_df, top_error_df

def save_video(save_file, tmp_dir, framerate, frame_pattern='frame_%06d.jpeg'):
    call_str = f'ffmpeg -r {framerate} -i {os.path.join(tmp_dir, frame_pattern)} -c:v libx264 -vf "pad=ceil(iw/2)*2:ceil(ih/2)*2" {save_file}'
    if os.name == 'nt':  # If the OS is Windows
        subprocess.run(['ffmpeg', '-r', str(framerate), '-i', f'{tmp_dir}/frame_%06d.jpeg',
                        '-c:v', 'libx264', '-vf', "pad=ceil(iw/2)*2:ceil(ih/2)*2",
                        save_file],
                       check=True)
    else:  # If the OS is Unix/Linux
        subprocess.run(['/bin/bash', '-c', call_str], check=True)

def plot_video_markers(markers_pd, ax, n, body_part, color, alphas, markers, model_id=0, markersize=8):
    x_key = body_part + '_x'
    y_key = body_part + '_y'
    markers_x = markers_pd[x_key][n]
    markers_y = markers_pd[y_key][n]
    ax.scatter(markers_x, markers_y, alpha=alphas[model_id], marker="o", color=color)

def read_error_files(relative_pixel_error_dir, specific_video=None):
    error_data = {}
    print(f"Reading error files from: {relative_pixel_error_dir}")
    print(f"Specific video filter: {specific_video}")

    for subdir, dirs, files in os.walk(relative_pixel_error_dir):
        for file in files:
            print(f"Found file: {file}")
            if file.endswith('.csv'):
                if specific_video and specific_video not in file:
                    print(f"Skipping file: {file}")
                    continue
                model_name = os.path.splitext(file)[0].replace('error_', '')
                error_df = pd.read_csv(os.path.join(subdir, file))
                error_data[model_name] = error_df
                print(f"Added file to error_data: {file}")

    return error_data


def expanding_mean(arr):
    return np.cumsum(arr) / np.arange(1, len(arr) + 1)

def organize_error_files(relative_pixel_error_dir):
    error_files = [f for f in os.listdir(relative_pixel_error_dir) if f.endswith('.csv')]
    for file in error_files:
        file_base_name = file.split('_')[1]
        sub_dir = os.path.join(relative_pixel_error_dir, file_base_name)
        if not os.path.exists(sub_dir):
            os.makedirs(sub_dir)
        os.rename(os.path.join(relative_pixel_error_dir, file), os.path.join(sub_dir, file))

def find_matching_video(videos_dir, base_file_name):
    for root, _, files in os.walk(videos_dir):
        for file in files:
            if file.startswith(base_file_name) and file.endswith(('.mp4', '.avi', '.mov')):
                return os.path.join(root, file)
    return None

In [27]:
def create_summed_ensemble_variance(networks_dir):
    data_type = 'lp'
    csv_name = "test_vid.csv"

    input_dfs, output_df, _ = format_data_walk(networks_dir, data_type, csv_name)
    print(f'Found {len(input_dfs)} models.')
    likelihood_thresh = 0.9
    likelihoods_above_thresh, summed_ensemble_vars, combined_df, bodypart_list = compute_likelihoods_and_variance(input_dfs, likelihood_thresh)
    print(summed_ensemble_vars)

    # Create a DataFrame from summed_ensemble_vars
    summed_ensemble_vars_df = pd.DataFrame(summed_ensemble_vars, columns=bodypart_list)

    # Define the output file path
    output_file_path = os.path.join(networks_dir, "summed_ensemble_vars.csv")

    # Save the DataFrame to a CSV file
    summed_ensemble_vars_df.to_csv(output_file_path, index=False)

    print(f'Saved summed_ensemble_vars to {output_file_path}')
    return output_file_path

In [28]:
def calculate_errors_main(eks_dir, aeks_eks_dir, output_dir, top_error_frame_count=20):
    eks_csv_files = get_csv_files(eks_dir)
    all_top_error_frames = {}
    all_error_dfs = []

    aeks_subdirs = [os.path.join(aeks_eks_dir, sub_dir) for sub_dir in os.listdir(aeks_eks_dir) if os.path.isdir(os.path.join(aeks_eks_dir, sub_dir))]

    top_error_frames_dir = os.path.join(output_dir, 'top_error_frames')
    relative_pixel_error_dir = os.path.join(output_dir, 'relative_pixel_error')

    # Ensure the directories exist
    if not os.path.exists(top_error_frames_dir):
        os.makedirs(top_error_frames_dir)
    if not os.path.exists(relative_pixel_error_dir):
        os.makedirs(relative_pixel_error_dir)

    for eks_file in eks_csv_files:
        eks_file_name = os.path.basename(eks_file)
        
        for sub_dir in aeks_subdirs:
            aeks_csv_files = get_csv_files(sub_dir)
            for aeks_file in aeks_csv_files:
                aeks_file_name = os.path.basename(aeks_file)
                
                if eks_file_name == aeks_file_name:
                    error_df, top_error_df = calculate_pixel_error(eks_file, aeks_file, top_error_frame_count)

                    base_file_name = os.path.splitext(eks_file_name)[0]  # Remove the original .csv extension if present
                    sub_dir_name = os.path.basename(sub_dir) if 'eks' not in os.path.basename(sub_dir) else 'aeks_eks'

                    # Save the error CSV to the relative_pixel_error directory
                    error_output_file = os.path.join(relative_pixel_error_dir, f"error_{base_file_name}_{sub_dir_name}.csv")
                    error_df.to_csv(error_output_file, index=False)
                    short_path = '/'.join(error_output_file.split('/')[-3:])
                    print(f"Pixel error calculated and saved to {short_path}")

                    # Create a subdirectory for the top_error_frames for the current file
                    top_error_frames_subdir = os.path.join(top_error_frames_dir, f"{base_file_name}_{sub_dir_name}")
                    if not os.path.exists(top_error_frames_subdir):
                        os.makedirs(top_error_frames_subdir)

                    # Save top error frames CSV to the subdirectory
                    top_error_frames_file = os.path.join(top_error_frames_subdir, f"top_error_frames_{base_file_name}_{sub_dir_name}.csv")
                    top_error_df.to_csv(top_error_frames_file, index=False)
                    print(f"Top error frames saved to {short_path}")

                    all_top_error_frames[f"{base_file_name}_{sub_dir_name}"] = top_error_df
                    all_error_dfs.append((error_df, sub_dir_name))

    return all_top_error_frames, all_error_dfs


In [29]:
def plot_video_frames(eks_dir, aeks_dir, output_dir, videos_dir, n_frames):
    eks_csv_files = get_csv_files(eks_dir)
    aeks_subdirs = get_subdirectories(aeks_dir)

    for eks_file in eks_csv_files:
        file_name = os.path.basename(eks_file)
        base_file_name = os.path.splitext(file_name)[0]  # Remove file extension

        video_path = find_matching_video(videos_dir, base_file_name)
        if not video_path:
            print(f"No matching video found for {file_name}")
            continue

        # Create a parent directory for the output videos for this eks_file
        parent_output_dir = os.path.join(output_dir, f'{base_file_name}_plotted_vids')
        if not os.path.exists(parent_output_dir):
            os.makedirs(parent_output_dir)

        for sub_dir in aeks_subdirs:
            aeks_csv_files = get_csv_files(sub_dir)
            matching_aeks_files = [f for f in aeks_csv_files if os.path.basename(f) == file_name]

            if not matching_aeks_files:
                print(f"No matching AEKS file found for {file_name} in {sub_dir}")
                continue

            aeks_file = matching_aeks_files[0]

            # Create a directory for the output videos inside the parent directory
            output_subdir = os.path.join(parent_output_dir, os.path.basename(sub_dir))
            if not os.path.exists(output_subdir):
                os.makedirs(output_subdir)

            # Load EKS
            markers_curr = pd.read_csv(eks_file, header=[0, 1, 2], index_col=0)
            keypoint_names = [c[1] for c in markers_curr.columns[::3]]
            model_name = markers_curr.columns[0][0]
            eks_pd = convert_lp_dlc(markers_curr, keypoint_names, model_name)

            # Load AEKS
            markers_curr = pd.read_csv(aeks_file, header=[0, 1, 2], index_col=0)
            keypoint_names = [c[1] for c in markers_curr.columns[::3]]
            model_name = markers_curr.columns[0][0]
            eks_pd2 = convert_lp_dlc(markers_curr, keypoint_names, model_name)

            # Setup for plotting
            animal_ids = [1]
            body_parts = ['tailMid_bot']  # Add other body parts as needed
            to_plot = []
            for animal_id in animal_ids:
                for body_part in body_parts:
                    to_plot.append(body_part)

            cap = cv2.VideoCapture(video_path)

            start_frame = 0
            idxs = np.arange(start_frame, start_frame + n_frames)
            framerate = 20

            colors = ['cyan', 'pink']
            alphas = [.8, 1.0]
            markers = ['.', 'x']
            model_labels = ['AEKS', 'EKS']
            model_colors = colors

            for body_part in to_plot:
                fig, ax = plt.subplots(1, 1, figsize=(10, 10))
                tmp_dir = os.path.join(output_subdir, f'tmp_{body_part}')
                if not os.path.exists(tmp_dir):
                    os.makedirs(tmp_dir)
                save_file = os.path.join(output_subdir, f'{base_file_name}_{body_part}.mp4')

                txt_fr_kwargs = {
                    'fontsize': 14, 'color': [1, 1, 1], 'horizontalalignment': 'left',
                    'verticalalignment': 'top', 'fontname': 'monospace',
                    'bbox': dict(facecolor='k', alpha=0.25, edgecolor='none'),
                    'transform': ax.transAxes
                }
                save_imgs = True
                markersize = 18 if save_imgs else 12
                for idx in tqdm(range(len(idxs))):
                    n = idxs[idx]
                    ax.clear()
                    frame = get_frames_from_idxs(cap, [n])
                    ax.imshow(frame[0, 0], vmin=0, vmax=255, cmap='gray')
                    ax.set_xticks([])
                    ax.set_yticks([])
                    patches = []

                    # AEKS
                    plot_video_markers(eks_pd2, ax, n, body_part, colors[0], alphas, markers, model_id=0, markersize=markersize)
                    # EKS
                    plot_video_markers(eks_pd, ax, n, body_part, colors[1], alphas, markers, model_id=1, markersize=markersize)

                    # Legend
                    for i, model_label in enumerate(model_labels):
                        patches.append(mpatches.Patch(color=model_colors[i], label=model_label))
                    ax.legend(handles=patches, prop={'size': 12}, loc='upper right')
                    im = ax.text(0.02, 0.98, f'frame {n}', **txt_fr_kwargs)
                    plt.savefig(os.path.join(tmp_dir, 'frame_%06d.jpeg' % idx))
                save_video(save_file, tmp_dir, framerate, frame_pattern='frame_%06d.jpeg')
                # Clean up temporary directory
                for file in os.listdir(tmp_dir):
                    os.remove(os.path.join(tmp_dir, file))
                os.rmdir(tmp_dir)

In [30]:
def plot_top_error_frames(video_path, eks_dir, aeks_eks_dir, output_dir, top_error_frames):
    cap = cv2.VideoCapture(video_path)
    
    for key in top_error_frames:
        base_file_name, sub_dir = key.split('_', 1)
        top_error_df = top_error_frames[key]
        subdirectory_name = f"{base_file_name}_{sub_dir}"
        
        top_error_frames_subdir = os.path.join(output_dir, 'top_error_frames', subdirectory_name)
        if not os.path.exists(top_error_frames_subdir):
            os.makedirs(top_error_frames_subdir)
        
        for _, row in top_error_df.iterrows():
            keypoint = row['keypoint']
            frame_idx = row['frame']
            error = row['error']
            
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            
            if not ret:
                print(f"Failed to read frame {frame_idx}")
                continue
            
            fig, ax = plt.subplots(1, 1, figsize=(10, 10))
            ax.imshow(frame, vmin=0, vmax=255)
            ax.set_title(f"Frame {frame_idx} - {keypoint} - Error: {error:.2f}")
            ax.axis('off')

            output_file = os.path.join(top_error_frames_subdir, f"frame_{frame_idx}_{keypoint}.png")
            plt.savefig(output_file, bbox_inches='tight')
            plt.close()
            print(f"Saved overlay frame plot to {output_file}")
    
    cap.release()

In [31]:
def plot_pixel_error_vs_ensemble_variance_from_files(error_data, ensemble_variance_csv, output_dir, specific_video=None, excluded_keypoints=None):
    ensemble_variance_df = pd.read_csv(ensemble_variance_csv)
    
    if excluded_keypoints is None:
        excluded_keypoints = []

    def plot_filtered_error_data(filtered_error_data, title_suffix):
        plt.figure(figsize=(10, 6))
        
        aeks_eks_plotted = False
        aeks_plotted = False

        for model_name, error_df in filtered_error_data.items():
            combined_errors = []
            combined_variance = []
            keypoints = error_df.columns

            for keypoint in keypoints:
                if keypoint in excluded_keypoints:
                    continue

                errors = error_df[keypoint].dropna()
                ensemble_variance = ensemble_variance_df[keypoint].dropna()

                if len(errors) == 0 or len(ensemble_variance) == 0:
                    continue

                combined_errors.extend(errors)
                combined_variance.extend(ensemble_variance)

            if len(combined_errors) == 0 or len(combined_variance) == 0:
                continue

            combined_errors = np.array(combined_errors)
            combined_variance = np.array(combined_variance)

            sorted_indices = np.argsort(combined_variance)
            sorted_combined_variance = combined_variance[sorted_indices]
            sorted_combined_errors = combined_errors[sorted_indices]
            avg_sorted_combined_errors = expanding_mean(sorted_combined_errors)

            # Define colors and labels
            if 'aeks_eks' in model_name:
                color = 'cyan'
                label = f'{title_suffix}_aeks_eks' if not aeks_eks_plotted else None
                aeks_eks_plotted = True
            else:
                color = 'gray'
                label = f'{title_suffix}_aeks' if not aeks_plotted else None
                aeks_plotted = True

            plt.plot(sorted_combined_variance, avg_sorted_combined_errors, label=label, color=color)

        plt.xlabel('Cumulative Ensemble Variance')
        plt.ylabel('Average Pixel Error')
        plt.title(f'Pixel Error Between EKS and aEKS models vs Ensemble Variance {title_suffix}')
        plt.grid(True)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

        combined_plot_output_file = os.path.join(output_dir, f'pixel_error_vs_ensemble_variance_{title_suffix}.png')
        plt.savefig(combined_plot_output_file, bbox_inches='tight')
        plt.close()
        print(f'Saved combined plot for all keypoints to {combined_plot_output_file}')

    # Filter and plot for test_vid
    test_vid_error_data = {k: v for k, v in error_data.items() if 'test_vid_new' not in k}
    plot_filtered_error_data(test_vid_error_data, 'test_vid')

    # Filter and plot for test_vid_new
    test_vid_new_error_data = {k: v for k, v in error_data.items() if 'test_vid_new' in k}
    plot_filtered_error_data(test_vid_new_error_data, 'test_vid_new')

In [32]:
# Create the summed ensemble variance CSV
summed_ensemble_variance_csv_path = create_summed_ensemble_variance(networks_dir)
print(f'Summed ensemble variance CSV created at: {summed_ensemble_variance_csv_path}')

# Generate error files (Ensure these functions are defined and working correctly)
top_error_frames, all_error_dfs = calculate_errors_main(eks_dir, aeks_dir, output_dir, top_error_frame_count)

# Print debug information to ensure error files are generated
print(f'Top error frames: {top_error_frames}')

# Verify the error_data
specific_video = 'test_vid'  # or 'test_vid_new'

# Read the error files
error_data = read_error_files(relative_pixel_error_dir, specific_video)

# Print the structure of error_data
print("Structure of error_data:", type(error_data), error_data.keys() if isinstance(error_data, dict) else error_data)

# Inspect a sample error_df from error_data to ensure it is correctly structured
if error_data:
    sample_model_name = list(error_data.keys())[0]
    sample_error_df = error_data[sample_model_name]
    print(sample_error_df.head())
else:
    print("No error data found")

# Plot pixel error vs ensemble variance using the created CSV
plot_pixel_error_vs_ensemble_variance_from_files(error_data, summed_ensemble_variance_csv_path, output_dir, specific_video, excluded_keypoints)

# Plot video frames with high pixel error
plot_video_frames(eks_dir, aeks_dir, output_dir, videos_dir, n_frames)
# plot_top_error_frames(videos_dir, eks_dir, aeks_dir, output_dir, top_error_frames)

Found 4 models.
[[3.0494259e+01 3.9371475e+03 1.0107673e+04 ... 6.8302035e+00
  5.3483398e+03 5.6356344e+00]
 [1.5978910e+01 2.2861479e+04 1.0846607e+04 ... 9.2598991e+00
  3.9423276e+03 1.2469720e+01]
 [2.3358734e+02 2.5563227e+04 1.1916753e+04 ... 1.5365124e+01
  1.7560588e+04 3.9972148e+00]
 ...
 [6.2902962e+01 2.5231861e+04 3.8865962e+03 ... 3.2295330e+01
  2.6774691e+04 2.4191128e+03]
 [6.3382397e+01 2.3804400e+04 4.0170271e+03 ... 3.6847614e+01
  1.6602248e+04 5.4640779e+00]
 [2.1678370e+01 2.2654721e+04 4.0397856e+03 ... 2.7029829e+01
  1.6587936e+04 4.7413273e+00]]
Saved summed_ensemble_vars to /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/networks/summed_ensemble_vars.csv
Summed ensemble variance CSV created at: /teamspace/studios/this_studio/outputs/mirror-mouse/hand=100_pseudo=1000/networks/summed_ensemble_vars.csv
Pixel error calculated and saved to eks_aeks_pixel_diff/relative_pixel_error/error_test_vid_aeks_eks.csv
Top error frames saved to eks_

100%|██████████| 10/10 [00:02<00:00,  4.36it/s]
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtw